In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [6]:
df_epo_cleantech = pd.read_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/Claims - Dim Reduction, Cluster/g_patent_claims_cleantech_yake_keywords_clusters_patentsberta.json')